Utworzyć ramkę danych o produktach farmaceutycznych zawierających dany lek (substancję leczniczą). Ramka powinna zawierać informacje o:
- ID leku,
- nazwie produktu,
- producencie,
- kod w narodowym rejestrze USA (ang. *National Drug Code*),
- postać w jakiej produkt występuje,
- sposób aplikacji,
- informacje o dawce,
- informacje kraju i agencji rejestrującej produkt.

In [56]:
import pandas as pd
import lxml
import xml.etree.ElementTree as ET

In [57]:
path = 'data/drugbank_partial.xml'
ns = {'db': 'http://www.drugbank.ca'}

In [58]:
tree = ET.parse(path)
root = tree.getroot()

In [59]:
def product_df(drug_id):
    data = []
    drug = root.find(f"db:drug[db:drugbank-id='{drug_id}']", ns)

    for product in drug.findall('db:products/db:product', ns):

        def unwrap(field, node: ET.Element=product):
            return getattr(node.find(f"db:{field}", ns), "text", None)
        
        data.append({
            "Product Name": unwrap("name"),
            "Labeller": unwrap("labeller"),
            "NDC ID": unwrap("ndc-id"),
            "NDC code": unwrap("ndc-product-code"),
            "Dosage Form": unwrap("dosage-form"),
            "Route": unwrap("route"),
            "Strength": unwrap("strength"),
            "Country": unwrap("country"),
            "Source": unwrap("source"),
        })

    return pd.DataFrame(data)

In [64]:
product_df("DB00091")

,Product Name,Labeller,NDC ID,NDC code,Dosage Form,Route,Strength,Country,Source
0,Apo-cyclosporine Oral Solution,Apotex Corporation,None,None,Solution,Oral,100 mg / mL,Canada,DPD
1,Aqua-stasis,Dr Marc's Manufacturing And Sales,None,70414-201,Liquid,Ophthalmic,0.0005 g/100mL,US,FDA NDC
2,Cequa,"Sun Pharmaceutical Industries, Inc.",None,47335-506,Solution / drops,Ophthalmic; Topical,0.0009 g/1mL,US,FDA NDC
3,Cequa,"Sun Pharmaceutical Industries, Inc.",None,47335-507,Solution / drops,Ophthalmic; Topical,0.0009 mg/1mL,US,FDA NDC
4,Cequa,Sun Pharmaceutical Industries Limited,None,None,Solution,Ophthalmic,0.09 % w/v,Canada,DPD
...,...,...,...,...,...,...,...,...,...
106,Verkazia,Santen Oy,None,None,Emulsion,Ophthalmic,1 mg/ml,EU,EMA
107,Verkazia,Santen Oy,None,None,Emulsion,Ophthalmic,1 mg/ml,EU,EMA
108,Verkazia,Santen Oy,None,None,Emulsion,Ophthalmic,1 mg/ml,EU,EMA
109,Vevye,Novaliq GmbH,None,83339-002,Solution / drops,Ophthalmic,1 mg/1mL,US,FDA NDC
